# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:
import pandas as pd
from decimal import Decimal
document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('name','infant_mortality'))
i = 0
for element in document.iterfind('country'):
    mortality = element.find('infant_mortality')
    if mortality is not None:
        #print(element.find('name').text, mortality.text)
        row = dict(zip(['name', 'infant_mortality'], [element.find('name').text, 
                                                      Decimal(element.find('infant_mortality').text)]))
    
    if mortality is None:    
        #print(element.find('name').text, 0.00)
        row = dict(zip(['name', 'infant_mortality'], [element.find('name').text, 0.00]))

    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    i = i + 1

infantmortality = df[df['infant_mortality'] != 0]
infantmortality.sort_values('infant_mortality').head(10)

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.6
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [147]:
import pandas as pd
from decimal import Decimal
document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('name','year', 'census_population'))
i = 0
for allcountry in document.iterfind('country'):
    for allcity in allcountry.findall('city'):
        city = allcity.find('name')
        if city is None:
            continue
        cityname = city.text
        population = allcity.findall('population')
        for allpopulation in population:
            if 'measured' in allpopulation.attrib:
                if allpopulation.get('measured') == 'census':
                    year = allpopulation.get('year')
                    totalpopulation = allpopulation.text
                    row = dict(zip(['name', 'year', 'census_population'], [cityname, int(year), int(totalpopulation)]))
                    
                    row_s = pd.Series(row)
                    row_s.name = i
                    df = df.append(row_s)
                    i = i + 1

citypopulation = df[df['census_population'] != 0]
citypopulation.sort_values('census_population', ascending = False).head(10)
citypopulation.groupby('name').max().sort_values('census_population', ascending = False).head(10)

,year,census_population
name,,
Seoul,2010,9895217
Al Qahirah,2006,8471859
Ho Chi Minh,2009,5968384
Singapore,2010,5076700
Al Iskandariyah,2006,4123869
Busan,2010,3662884
Pyongyang,2008,3255288
Nairobi,2009,3133518
Santo Domingo,2010,2749703


In [148]:
import pandas as pd
from decimal import Decimal

document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('ethnic_group','total_population'))
i = 0
for allcountry in document.iterfind('country'):
    maxpopulation = [0]
    for allpopulation in allcountry.findall("population"):
        if 'measured' in allpopulation.attrib:
            if allpopulation.get('measured') != 'est.':
                maxpopulation.append(0)
            if allpopulation.get('measured') == 'est.':
                maxpopulation.append(int(allpopulation.text))
                #print(maxpopulation)
                #print(allcountry.find("name").text, allpopulation.get('measured'), allpopulation.get('year'), allpopulation.text)
    for allethnicgroup in allcountry.findall("ethnicgroup"):
        if allethnicgroup is None:
            name = ""
            populationpercentage = 0
            continue
        if allethnicgroup is not None:
            name = allethnicgroup.text
            populationpercentage = Decimal(allethnicgroup.get('percentage'))
            #print(name, populationpercentage)
            #print(allcountry.find("name").text, name, (max(maxpopulation)*populationpercentage)/100) 
            
            row = dict(zip(['ethnic_group', 'total_population'], [name, (max(maxpopulation)*populationpercentage)/100]))
                    
            row_s = pd.Series(row)
            row_s.name = i
            df = df.append(row_s)
            i = i + 1
df

populationbyethnicgroup = df[df['total_population'] != 0]
populationbyethnicgroup.sort_values('total_population', ascending = False).head(10)
populationbyethnicgroup.groupby('ethnic_group').max().sort_values('total_population', ascending = False).head(10)

,total_population
ethnic_group,
Han Chinese,1066367504.805
European,203503514.9412
Japanese,124960385.956
Russian,118222704.096
African,94661176.5
Viet/Kinh,59055769.731
Mulatto,57614611.285
Mestizo,51646202.4
English,47831303.608


In [87]:
import pandas as pd
from decimal import Decimal
document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('river_name','country_name', 'river_length'))
i = 0
for element in document.findall('river'):
    river_name = element.find('name').text
    country_name = element.get('country')
    
    #print(river_name, country_name, river_length)
    if element.find('length') is None:
        river_length = 0.00    
        continue
    if element.find('length') is not None:
        river_length = element.find('length').text
    row = dict(zip(['river_name', 'country_name', 'river_length'], [river_name, country_name, Decimal(river_length)]))
                    
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    i = i + 1

df
allrivers = df[df['river_length'] != 0.0]
allrivers.groupby('river_name').max().sort_values('river_length', ascending = False).head(1)

,country_name,river_length
river_name,,
Amazonas,CO BR PE,6448


In [88]:
import pandas as pd
from decimal import Decimal
document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('lake_name','country_name', 'lake_area'))
i = 0
for element in document.findall('lake'):
    river_name = element.find('name').text
    country_name = element.get('country')
    
    #print(river_name, country_name, river_length)
    if element.find('area') is None:
        river_length = 0.00    
        continue
    if element.find('area') is not None:
        river_length = element.find('area').text
    row = dict(zip(['lake_name', 'country_name', 'lake_area'], [river_name, country_name, Decimal(river_length)]))
                    
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    i = i + 1

df
allrivers = df[df['lake_area'] != 0.0]
allrivers.groupby('lake_name').max().sort_values('lake_area', ascending = False).head(1)

,country_name,lake_area
lake_name,,
Caspian Sea,R AZ KAZ IR TM,386400


In [107]:
import pandas as pd
from decimal import Decimal

document = ET.parse( './data/mondial_database.xml' )

df = pd.DataFrame(columns=('airport_name','country_name', 'airport_elevation'))
i = 0
for element in document.findall('airport'):
    airport_name = element.find('name').text
    country_name = element.get('country')
    #print(airport_name, country_name)
    if element.find('elevation') is None:
        airport_elevation = 0
        continue
    if element.find('elevation') is not None:
        airport_elevation = element.find('elevation').text
    row = dict(zip(['airport_name', 'country_name', 'airport_elevation'], [airport_name, country_name, airport_elevation]))
                    
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    i = i + 1

df
allrivers = df[df['airport_elevation'] != 0.0]
allrivers.groupby('airport_name').max().sort_values('airport_elevation', ascending = False).head(1)

,country_name,airport_elevation
airport_name,,
Mashhad,IR,995
